In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.


In [ ]:
# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/gemma-7b-it-bnb-4bit", # Instruct version of Gemma 7b
    "unsloth/gemma-2b-bnb-4bit",
    "unsloth/gemma-2b-it-bnb-4bit", # Instruct version of Gemma 2b
    "unsloth/llama-3-8b-bnb-4bit", # [NEW] 15 Trillion token Llama-3
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

In [4]:
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Driseri/lora_model_llama3-8B-good_1", # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference


/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/732 [00:00<?, ?B/s]

Unsloth: You passed in `unsloth/llama-3-8b-bnb-4bit` which is a 4bit model, yet you set
`load_in_4bit = False`. We shall load `unsloth/llama-3-8b` instead.


==((====))==  Unsloth: Fast Llama patching release 2024.4
   \\   /|    GPU: NVIDIA RTX A5000. Max memory: 23.679 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.0. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


adapter_model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Unsloth 2024.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [5]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

In [8]:

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
        """// This is a module that implements an AND gate
module and_gate( 
input a, 
input b, 
output out );
// assign the AND of a and b to out"""
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


['<|begin_of_text|>// This is a module that implements an AND gate\nmodule and_gate( \ninput a, \ninput b, \noutput out );\n// assign the AND of a and b to out\nassign out = a & b;\nendmodule\n<|end_of_text|>']

In [20]:
def makecode(prompt): # Enable native 2x faster inference
    inputs = tokenizer(
    [
    prompt
    ], return_tensors = "pt").to("cuda")
    
    outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True, temperature = 0.01)
    output = tokenizer.batch_decode(outputs)
    return output

In [21]:
import os

# Задаем корневую директорию
root_dir = 'Vgen'
content = ''

# Проходим по всем директориям и файлам в корневой директории
for subdir, dirs, files in os.walk(root_dir):
    num = 0
    for file in files:
        # Проверяем, содержит ли имя файла 'prompt'
        if 'prompt' in file:
            
            print('++++++++++')
            print(file)
            print(subdir)
            if subdir == 'Vgen/basic1/.ipynb_checkpoints':
                continue
            
            num = num + 1
            # Формируем полный путь к файлу
            file_path = os.path.join(subdir, file)
            #print(file)
            # Читаем содержимое файла и добавляем его в переменную content
            #print(file_path)
            #print(file_path.replace('Vgen/', "").replace('/'+file,''))
            #vfile = file_path.replace('Vgen/', "").replace('/'+file,'')
            with open(file_path, 'r') as file:
                content = file.read()
            output = makecode(content)
            print((subdir[5:]+'_'+str(num)).strip('/'))
            with open(subdir[5:]+'_'+str(num), 'w') as file:
                file.write(output[0])
                

# Указываем путь и имя файла, в который будет сохранен результат
output_file_path = 'output.txt'
# Записываем содержимое в файл
#with open(output_file_path, 'w') as output_file:
#    output_file.write(content)

#print(f"Содержимое сохранено в файл: {output_file_path}")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


++++++++++
prompt1_truthtable.v
Vgen/intermediate8


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


intermediate8_1
++++++++++
prompt2_truthtable.v
Vgen/intermediate8


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

advanced2_3
++++++++++
prompt1_signed-addition-overflow.v
Vgen/advanced1


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


advanced1_1
++++++++++
prompt2_signed-addition-overflow.v
Vgen/advanced1


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


advanced1_2
++++++++++
prompt3_signed-addition-overflow.v
Vgen/advanced1
advanced1_3
